## 5.5 Betting Against Betaの検証

### コード 5.4 必要なライブラリの読み込み

In [1]:
import os
import pandas as pd
import statsmodels.api as sm
os.makedirs('./output', exist_ok=True)

### コード5.5 ベータ推定用のデータセットを作成するプログラム

In [2]:
# 銘柄ごとの日次リターンデータの読み込みと、計算に必要な列の選択
stockDaily = pd.read_csv('./data/stockDaily.csv', parse_dates=['date'])
stockDaily['date'] = stockDaily['date'].dt.to_period('D')
stockDaily = stockDaily[['ticker', 'date', 'return']]
print(stockDaily)
##          ticker       date    return
## 0         A0001 1991-01-04 -0.884354
## 1         A0001 1991-01-07 -0.068634

# 日次データをベースにしたβの推定のためffDailyの読み込み
ffDaily = pd.read_csv('./data/ffDaily.csv', parse_dates=['date'])
ffDaily['date'] = ffDaily['date'].dt.to_period('D')
print(ffDaily)
##             date  RMRF   SMB   HML    RF
## 0     1990-07-02 -0.08  1.46 -0.73  0.03
## 1     1990-07-03 -0.40  0.65 -0.41  0.03

# ffMonthlyのRMRF,RFを結合し，銘柄ごとに超過リターン (RIRF = return - RF)の計算
df = stockDaily.merge(ffDaily[['date', 'RMRF', 'RF']], on='date')
df['RIRF'] = df['return'] - df['RF']
print(df)
##          ticker        date    return  RMRF    RF      RIRF
## 0         A0001  1991-01-04 -0.884354 -0.49  0.02 -0.904354
## 1         A0004  1991-01-04 -1.454469 -0.49  0.02 -1.474469

# 日付に対応する月をセットする
df['month'] = df['date'].dt.asfreq('M')

# 必要な列を切り出してデータセットの完成
df = df[['month', 'ticker', 'date', 'RIRF', 'RMRF']]
print(df)
##             month ticker        date      RIRF  RMRF
## 0         1991-01  A0001  1991-01-04 -0.904354 -0.49
## 1         1991-01  A0004  1991-01-04 -1.474469 -0.49

         ticker        date    return
0         A0001  1991-01-04 -0.884354
1         A0001  1991-01-07 -0.068634
2         A0001  1991-01-08 -1.030220
3         A0001  1991-01-09 -0.277585
4         A0001  1991-01-10 -0.904663
...         ...         ...       ...
12005999  Z0137  2014-12-24  2.380952
12006000  Z0137  2014-12-25 -0.310078
12006001  Z0137  2014-12-26  0.311042
12006002  Z0137  2014-12-29 -2.170543
12006003  Z0137  2014-12-30  2.535658

[12006004 rows x 3 columns]
            date  RMRF   SMB   HML    RF
0     1990-07-02 -0.08  1.46 -0.73  0.03
1     1990-07-03 -0.40  0.65 -0.41  0.03
2     1990-07-04  1.49 -0.01  0.32  0.03
3     1990-07-05  0.16  2.71 -0.12  0.03
4     1990-07-06  1.17  0.74 -0.03  0.03
...          ...   ...   ...   ...   ...
6144  2015-06-24  1.05 -0.50 -0.15  0.00
6145  2015-06-25  0.61 -0.04 -0.38  0.00
6146  2015-06-26  0.92 -0.16  0.11  0.00
6147  2015-06-29  0.05 -0.16  0.01  0.00
6148  2015-06-30 -0.87 -0.04  0.17  0.00

[6149 rows x 5 columns

### コード 5.6 過去12ヶ月の日次データから銘柄別のベータを推定するプログラム

In [4]:
# 過去12ヶ月の日次データから銘柄別のβを計算するプログラム

# 単回帰でβと観測値数を計算する関数
def ols(d):
    model = sm.OLS(d['RIRF'], sm.add_constant(d['RMRF']))  # (5.3)式の回帰モデル
    res = model.fit()  # OLSによる推定
    beta = res.params['RMRF']  # 回帰係数(β) (numpy.float64)
    nobs = res.nobs  # 観測値数 (float)
    # 推定されたβと観測値数をpandasのSeriesにして返す
    return pd.Series([beta, nobs], index=['beta', 'nobs'])


# 月次のリスト
# 推定は2010年1月から2014年11月までの各月において過去12ヶ月のデータを利用して推定
mrange = pd.period_range('2010-01', '2014-11', freq='M')
betas = []
for t in mrange:
    print('## month %s START' % (t))
    tp1 = t + 1  # tp1に月次t+1をセット
    tm11 = t - 11  # tm11に月次t-11をセット
    # 月次t-11から月次tまでの12ヶ月をdataとしてセット
    data = df[(tm11 <= df['month']) & (df['month'] <= t)]
    # tickerごとに回帰モデルを推定し，βと観測値数を得る
    result = data.groupby('ticker').apply(ols)
    # 観測値数が120営業日以上ものだけに限定
    result = result[result['nobs'] >= 120]
    ##             beta   nobs
    ## ticker
    ## A0001   1.185982  243.0
    ## A0002   0.593264  243.0

    # 各銘柄が運用期間である月次t+1にどの10分位に属するかを算定
    # βをrank関数で順位に変換しているのは、qcut関数に与える数値列に重複があってはならないため、
    # 同じβの値があれば、それを出現順に順位を付与し、その後に10分位を求めている。
    labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    result['decile'] = pd.qcut(result['beta'].rank(method='first'),
                               q=10, labels=labels).rename('decile')

    # 全行に運用期間である月次t+1をmonthとして挿入
    result.insert(0, 'month', tp1)
    ##             month      beta   nobs decile
    ## ticker
    ## A0001  2010-02-01  1.185982  243.0     10
    ## A0002  2010-02-01  0.593264  243.0      7
    betas.append(result)

betas = pd.concat(betas)  # 月別に計算したbeta, nobs, decileのDataFrameを縦方向に連結
betas.to_csv('./output/betas.csv')  # betas.csvとしてoutputフォルダに出力
print(betas)
##          month      beta   nobs decile
## ticker
## A0001   2010-02  1.185982  243.0     10
## A0002   2010-02  0.593264  243.0      7

## month 2010-01 START
## month 2010-02 START
## month 2010-03 START
## month 2010-04 START
## month 2010-05 START
## month 2010-06 START
## month 2010-07 START
## month 2010-08 START
## month 2010-09 START
## month 2010-10 START
## month 2010-11 START
## month 2010-12 START
## month 2011-01 START
## month 2011-02 START
## month 2011-03 START
## month 2011-04 START
## month 2011-05 START
## month 2011-06 START
## month 2011-07 START
## month 2011-08 START
## month 2011-09 START
## month 2011-10 START
## month 2011-11 START
## month 2011-12 START
## month 2012-01 START
## month 2012-02 START
## month 2012-03 START
## month 2012-04 START
## month 2012-05 START
## month 2012-06 START
## month 2012-07 START
## month 2012-08 START
## month 2012-09 START
## month 2012-10 START
## month 2012-11 START
## month 2012-12 START
## month 2013-01 START
## month 2013-02 START
## month 2013-03 START
## month 2013-04 START
## month 2013-05 START
## month 2013-06 START
## month 2013-07 START
## month 20

,month,beta,nobs,decile
ticker,,,,
A0001,2010-02,1.185982,243.0,10
A0002,2010-02,0.593264,243.0,7
A0003,2010-02,0.690904,243.0,8
A0004,2010-02,0.731857,243.0,8
A0005,2010-02,0.786377,243.0,8
...,...,...,...,...
Z0133,2014-12,0.282793,243.0,2
Z0134,2014-12,0.689158,243.0,5
Z0135,2014-12,0.341998,243.0,3


### コード5.7 ポートフォリオを構築するプログラム

In [5]:
# 銘柄ごとの月次リターンデータの読み込みと、Jensenのアルファの推定に必要な列の選択
stockMonthly = pd.read_csv('./data/stockMonthly.csv', parse_dates=['month'])
stockMonthly['month'] = stockMonthly['month'].dt.to_period('M')
stockMonthly = stockMonthly[['ticker', 'month', 'return']]
print(stockMonthly)
##        ticker    month     return
## 0       A0001  1991-01   4.285714
## 1       A0001  1991-02  -1.956947

# 市場ポートフォリオの超過リターンが収録されたffMonthlyを読み込み
ffMonthly = pd.read_csv('./data/ffMonthly.csv', parse_dates=['month'])
ffMonthly['month'] = ffMonthly['month'].dt.to_period('M')
print(ffMonthly)
##        month   RMRF   SMB   HML    RF
## 0    1990-07  20.67 -1.56 -5.16  0.68
## 1    1990-08 -13.69 -3.63  0.98  0.66

# stockMonthlyとbetasとtickerとmonthをキーにして結合し，それをbabとする
bab = betas.merge(stockMonthly, on=['ticker', 'month'])
print(bab)
##        ticker    month      beta   nobs decile     return
## 0       A0001  2010-02  1.185982  243.0     10   3.583427
## 1       A0002  2010-02  0.593264  243.0      7  12.535860

# 各月ごとにポートフォリオごとに等加重平均リターンを算定
bab = bab[['month', 'decile', 'return']].groupby(['month', 'decile']).mean()
bab = bab.reset_index()
print(bab)
##       month decile    return
## 0    2010-02      1 -1.578667
## 1    2010-02      2 -1.555539
## ..       ...    ...       ...
## 588  2014-12      9  7.270449
## 589  2014-12     10  7.059947

# ffMonthlyのRMRF,RFを結合し，被説明変数になる各ポートフォリオの超過リターン (RPRF = return - RF)を計算
bab = bab.merge(ffMonthly[['month', 'RMRF', 'RF']], on='month')
bab['RPRF'] = bab['return'] - bab['RF']
print(bab)
##        month decile    return  RMRF   RF      RPRF
## 0    2010-02      1 -1.578667 -1.99  0.0 -1.578667
## 1    2010-02      2 -1.555539 -1.99  0.0 -1.555539
## ..       ...    ...       ...   ...  ...       ...
## 588  2014-12      9  7.270449  8.86  0.0  7.270449
## 589  2014-12     10  7.059947  8.86  0.0  7.059947

       ticker    month     return
0       A0001  1991-01   4.285714
1       A0001  1991-02  -1.956947
2       A0001  1991-03   3.060546
3       A0001  1991-04 -10.329240
4       A0001  1991-05  -8.927286
...       ...      ...        ...
586021  Z0137  2014-08  -0.725953
586022  Z0137  2014-09   3.107861
586023  Z0137  2014-10   1.773050
586024  Z0137  2014-11  -3.310105
586025  Z0137  2014-12  16.576580

[586026 rows x 3 columns]
       month   RMRF   SMB   HML    RF
0    1990-07  20.67 -1.56 -5.16  0.68
1    1990-08 -13.69 -3.63  0.98  0.66
2    1990-09   0.63  5.18  1.83  0.60
3    1990-10  -3.76 -5.30  1.84  0.68
4    1990-11 -19.52 -0.50  1.43  0.57
..       ...    ...   ...   ...   ...
295  2015-02   4.59 -2.00  0.73  0.00
296  2015-03   8.81 -4.46  0.11  0.00
297  2015-04   0.99 -0.16 -1.50  0.00
298  2015-05   4.60 -0.98 -0.49  0.00
299  2015-06   2.66  0.49 -0.74  0.00

[300 rows x 5 columns]
       ticker    month      beta   nobs decile     return
0       A0001  2010-02  1.1

,month,decile,return
0,2010-02,1,-1.578667
1,2010-02,2,-1.555539
2,2010-02,3,-2.042178
3,2010-02,4,-1.981869
4,2010-02,5,-3.928856
...,...,...,...
585,2014-12,6,5.371436
586,2014-12,7,5.632818
587,2014-12,8,6.908590
588,2014-12,9,7.270449


       month decile    return  RMRF   RF      RPRF
0    2010-02      1 -1.578667 -1.99  0.0 -1.578667
1    2010-02      2 -1.555539 -1.99  0.0 -1.555539
2    2010-02      3 -2.042178 -1.99  0.0 -2.042178
3    2010-02      4 -1.981869 -1.99  0.0 -1.981869
4    2010-02      5 -3.928856 -1.99  0.0 -3.928856
..       ...    ...       ...   ...  ...       ...
585  2014-12      6  5.371436  8.86  0.0  5.371436
586  2014-12      7  5.632818  8.86  0.0  5.632818
587  2014-12      8  6.908590  8.86  0.0  6.908590
588  2014-12      9  7.270449  8.86  0.0  7.270449
589  2014-12     10  7.059947  8.86  0.0  7.059947

[590 rows x 6 columns]


### コード5.8 CAPMによりポートフォリオを評価するプログラム

In [6]:
def ols(d):
    # 各ポートフォリオのCAPMアルファを推定するための回帰モデル
    model = sm.OLS(d['RPRF'], sm.add_constant(d[['RMRF']]))
    res = model.fit()
    coef = res.params  # 偏回帰係数
    coef.index = ['alpha', 'RMRF']
    pval = res.pvalues  # 各説明変数のp値
    pval.index = ['alpha_p', 'RMRF_p']
    return pd.concat([coef, pval])


# ポートフォリオごとに回帰モデルを推定し，babAlphaとする
babAlpha = bab.groupby('decile').apply(ols)
print(babAlpha)
##           alpha      RMRF   alpha_p        RMRF_p
## decile
## 1       0.636046  0.260333  0.006948  3.914043e-07
## 2       0.730353  0.390996  0.005239  1.629348e-10

           alpha      RMRF   alpha_p        RMRF_p
decile                                            
1       0.636046  0.260333  0.006948  3.914043e-07
2       0.730353  0.390996  0.005239  1.629348e-10
3       0.730063  0.510106  0.008749  2.484246e-13
4       0.917976  0.650195  0.001732  1.050829e-16
5       0.701263  0.748830  0.027560  2.384770e-17
6       0.655439  0.838360  0.035290  8.187064e-20
7       0.504025  0.982350  0.112090  2.123780e-22
8       0.502792  1.095347  0.142313  5.214836e-23
9       0.413293  1.244935  0.197272  4.601612e-27
10     -0.056145  1.469226  0.882761  9.076405e-27


,alpha,RMRF,alpha_p,RMRF_p
decile,,,,
1,0.636046,0.260333,0.006948,3.914043e-07
2,0.730353,0.390996,0.005239,1.629348e-10
3,0.730063,0.510106,0.008749,2.484246e-13
4,0.917976,0.650195,0.001732,1.050829e-16
5,0.701263,0.748830,0.027560,2.384770e-17
6,0.655439,0.838360,0.035290,8.187064e-20
7,0.504025,0.982350,0.112090,2.123780e-22
8,0.502792,1.095347,0.142313,5.214836e-23
9,0.413293,1.244935,0.197272,4.601612e-27


### コード5.9 ラッキーセブン戦略のFF3アルファを推定するプログラム

In [2]:
# 教科書ではコード5.2,5.3の修正箇所のみを示しているが、以下では、
# コード5.2〜5.3を再掲し、このセル単独で動作するようにしている。

import pandas as pd
import statsmodels.api as sm

# 必要なデータの選択(行と列の選択)
# ticker末尾が0007の 2011/7-2014/6の36ヶ月のmonthとreturn
stockMonthly = pd.read_csv('./data/stockMonthly.csv', parse_dates=['month'])
stockMonthly['month'] = stockMonthly['month'].dt.to_period('M')
stockMonthly = stockMonthly[stockMonthly['ticker'].str[1:5] == '0007']
stockMonthly = stockMonthly[(stockMonthly['month'] >= '2011-7') &
                            (stockMonthly['month'] <= '2014-6')]
stockMonthly = stockMonthly[['month', 'ticker', 'return']]
print(stockMonthly)
##          month ticker      return
## 1596    2011-07  A0007    0.598007
## 1597    2011-08  A0007    2.245707

# 市場ポートフォリオの超過リターンが収録されたffMonthlyを読み込み
ffMonthly = pd.read_csv('./data/ffMonthly.csv', parse_dates=['month'])
ffMonthly['month'] = ffMonthly['month'].dt.to_period('M')
print(ffMonthly)
##        month   RMRF   SMB   HML    RF
## 0    1990-07  20.67 -1.56 -5.16  0.68
## 1    1990-08 -13.69 -3.63  0.98  0.66


# 抽出されたラッキーセブン銘柄群の等加重平均リターン（ラッキーセブン戦略によるポートフォリオリターン）を算定
df = stockMonthly.groupby('month').mean()
# ffMonthlyのRMRF,RF，さらにSMBとHMLも結合(インデックスによる結合)
df = pd.merge(df, ffMonthly[['month', 'RMRF', 'RF', 'SMB', 'HML']], on='month')  # SMBとHMLもdfに結合
# ラッキーセブンポートフォリオの超過リターンを算定 (RPRF = return - RF)
df['RPRF'] = df['return'] - df['RF']
# 回帰モデル構築用のデータセットの完成
print(df)
##       month     return   RMRF    RF   SMB   HML       RPRF
## 0   2011-07  11.665931   2.02  0.00  0.76  2.10  11.665931
## 1   2011-08   7.900743   3.99  0.01  0.48  1.19   7.890743

# ラッキーセブン戦略によるFF3アルファの推定
model = sm.OLS(df['RPRF'], sm.add_constant(df[['RMRF', 'SMB', 'HML']]))
res = model.fit()  # OLSによる推定
print(res.summary())  # 推定結果を表示

          month ticker      return
1596    2011-07  A0007    0.598007
1597    2011-08  A0007    2.245707
1598    2011-09  A0007   -2.131783
1599    2011-10  A0007    4.818482
1600    2011-11  A0007   -3.211587
...         ...    ...         ...
561769  2014-02  Z0007  143.115500
561770  2014-03  Z0007   -7.559395
561771  2014-04  Z0007  -19.496360
561772  2014-05  Z0007   14.704930
561773  2014-06  Z0007   40.736580

[900 rows x 3 columns]
       month   RMRF   SMB   HML    RF
0    1990-07  20.67 -1.56 -5.16  0.68
1    1990-08 -13.69 -3.63  0.98  0.66
2    1990-09   0.63  5.18  1.83  0.60
3    1990-10  -3.76 -5.30  1.84  0.68
4    1990-11 -19.52 -0.50  1.43  0.57
..       ...    ...   ...   ...   ...
295  2015-02   4.59 -2.00  0.73  0.00
296  2015-03   8.81 -4.46  0.11  0.00
297  2015-04   0.99 -0.16 -1.50  0.00
298  2015-05   4.60 -0.98 -0.49  0.00
299  2015-06   2.66  0.49 -0.74  0.00

[300 rows x 5 columns]
      month     return   RMRF    RF   SMB   HML       RPRF
0   2011-07  11.6